In [218]:
from __future__ import print_function, division
import sys,os
# line 4 and line 5 below are for development purposes and can be removed
qspin_path = os.path.join(os.getcwd(),"../../")
sys.path.insert(0,qspin_path)
from scipy.linalg import expm
from quspin.operators import hamiltonian, commutator, exp_op # Hamiltonians and operators
from quspin.basis import tensor_basis, spin_basis_1d # bases
from quspin.tools.measurements import obs_vs_time # calculating dynamics
from quspin.tools.Floquet import Floquet_t_vec # period-spaced time vector
import numpy as np # general math functions
import matplotlib.pyplot as plt # plotting library

In [219]:
L = 3
M = 0.1
l=1
no_checks = dict(check_pcon=False,check_symm=False,check_herm=False)


In [220]:
basis=spin_basis_1d(L=L,S = str(l))
print(basis)

reference states: 
array index   /   Fock state   /   integer repr. 
      0.         |2 2 2>           26  
      1.         |2 2 1>           25  
      2.         |2 2 0>           24  
      3.         |2 1 2>           23  
      4.         |2 1 1>           22  
      5.         |2 1 0>           21  
      6.         |2 0 2>           20  
      7.         |2 0 1>           19  
      8.         |2 0 0>           18  
      9.         |1 2 2>           17  
     10.         |1 2 1>           16  
     11.         |1 2 0>           15  
     12.         |1 1 2>           14  
     13.         |1 1 1>           13  
     14.         |1 1 0>           12  
     15.         |1 0 2>           11  
     16.         |1 0 1>           10  
     17.         |1 0 0>            9  
     18.         |0 2 2>            8  
     19.         |0 2 1>            7  
     20.         |0 2 0>            6  
     21.         |0 1 2>            5  
     22.         |0 1 1>            4  
     23.   

In [221]:
basis_vectors = []
for index in range(basis.Ns):
    state = np.zeros(basis.Ns)
    state[basis.Ns-index-1] = 1.
    state = state.tolist()
    basis_vectors.append(state)

In [222]:
#Gauss_law_0

gauss_law_z_0 = []
gauss_law_z_0.append([1.,0])

gauss_law_0_map = [
    
    ["z",gauss_law_z_0]

    
]

Gauss_law_0  = hamiltonian(gauss_law_0_map,dynamic_list=[],basis=basis,**no_checks)

In [223]:
Proj_10 = 0
for state in basis_vectors:
    state = np.array(state)
    if (Gauss_law_0.expt_value(state)>=1.-1e-8) & (Gauss_law_0.expt_value(state)<=1.+1e-8):
        Proj_10 += np.outer(state,state)

In [224]:
#Gauss_law_1
gauss_law_zm_1 = []
gauss_law_zm_1.append([-1.,0])

gauss_law_z_1 = []
gauss_law_z_1.append([1.,1])

gauss_law_id_1 = [[1.,1]]

gauss_law_1_map = [
    
    ["z",gauss_law_z_1],
    ["z",gauss_law_zm_1],
    ["I",gauss_law_id_1]
    
]

Gauss_law_1  = hamiltonian(gauss_law_1_map,dynamic_list=[],basis=basis,**no_checks)

In [225]:
Proj_01 = 0
Proj_11 = 0
for state in basis_vectors:
    state = np.array(state)
    if (Gauss_law_1.expt_value(state)>=0.-1e-8) & (Gauss_law_1.expt_value(state)<=0.+1e-8):
        Proj_01 += np.outer(state,state)
    if (Gauss_law_1.expt_value(state)>=1.-1e-8) & (Gauss_law_1.expt_value(state)<=1.+1e-8):
        Proj_11 += np.outer(state,state)

In [226]:
#Gauss_law_2
gauss_law_zm_2 = []
gauss_law_zm_2.append([-1.,1])

gauss_law_z_2 = []
gauss_law_z_2.append([1.,2])


gauss_law_2_map = [
    
    ["z",gauss_law_z_2],
    ["z",gauss_law_zm_2],
    
]

Gauss_law_2  = hamiltonian(gauss_law_2_map,dynamic_list=[],basis=basis,**no_checks)

In [227]:
Proj_02 = 0
Proj_12 = 0
for state in basis_vectors:
    state = np.array(state)
    if (Gauss_law_2.expt_value(state)>=0.-1e-8) & (Gauss_law_2.expt_value(state)<=0.+1e-8):
        Proj_02 += np.outer(state,state)
    if (Gauss_law_2.expt_value(state)>=1.-1e-8) & (Gauss_law_2.expt_value(state)<=1.+1e-8):
        Proj_12 += np.outer(state,state)

In [228]:
#Gauss_law_3

gauss_law_z_3 = []
gauss_law_z_3.append([-1.,2])

gauss_law_id_3 = [[1.,2]]

gauss_law_3_map = [
    
    ["z",gauss_law_z_3],
    ["I",gauss_law_id_3]

    
]

Gauss_law_3  = hamiltonian(gauss_law_3_map,dynamic_list=[],basis=basis,**no_checks)

In [229]:
Proj_03 = 0
for state in basis_vectors:
    state = np.array(state)
    if (Gauss_law_3.expt_value(state)>=0.-1e-8) & (Gauss_law_3.expt_value(state)<=0.+1e-8):
        Proj_03 += np.outer(state,state)


In [236]:
quadratic_term_0 = [[2./np.sqrt(2),0,1]]
linear_term_0 = [[1./np.sqrt(2),0]]

linear_term_1 = [[-1./np.sqrt(2),1]]


linear_term_2 = [[1./np.sqrt(2),2]]

kin_energy = [[0.5,i,i] for i in range(L)]
mass_term = []
mass_term.append([2*M,0])
mass_term.append([-2*M,1])
mass_term.append([2*M,2])

kin_mass_map = [
            ["zz", kin_energy], 
            ["z", mass_term], 
]

interaction_map_p0 = [
    
    ["+z", quadratic_term_0],
    ["+", linear_term_0]
]

interaction_map_m0 = [
    
    ["-z", quadratic_term_0],
    ["-", linear_term_0]
]

interaction_map_p1 = [
    
    ["+", linear_term_1]
]

interaction_map_m1 = [
    
    ["-", linear_term_1]
]


interaction_map_p2 = [
    
    ["+", linear_term_2]
]

interaction_map_m2 = [
    
    ["-", linear_term_2]
]



H_int_p0 = hamiltonian(interaction_map_p0,dynamic_list=[],basis=basis,**no_checks)
H_int_m0 = hamiltonian(interaction_map_m0,dynamic_list=[],basis=basis,**no_checks)

H_int_p1 = hamiltonian(interaction_map_p1,dynamic_list=[],basis=basis,**no_checks)
H_int_m1 = hamiltonian(interaction_map_m1,dynamic_list=[],basis=basis,**no_checks)

H_int_p2 = hamiltonian(interaction_map_p2,dynamic_list=[],basis=basis,**no_checks)
H_int_m2 = hamiltonian(interaction_map_m2,dynamic_list=[],basis=basis,**no_checks)

H_kin_mass = hamiltonian(kin_mass_map,dynamic_list=[],basis=basis,**no_checks)

h_int_p0 = H_int_p0.toarray()
h_int_m0 = H_int_m0.toarray()

h_int_p1 = H_int_p1.toarray()
h_int_m1 = H_int_m1.toarray()

h_int_p2 = H_int_p2.toarray()
h_int_m2 = H_int_m2.toarray()

h_kin_mass = H_kin_mass.toarray()

h_int_0 = Proj_10 @ h_int_p0 @ Proj_01 + Proj_01 @ h_int_m0 @ Proj_10
h_int_1 = Proj_11 @ h_int_p1 @ Proj_02 + Proj_02 @ h_int_m1 @ Proj_11
h_int_2 = Proj_12 @ h_int_p2 @ Proj_03 + Proj_03 @ h_int_m2 @ Proj_12

h_int = h_int_0+h_int_1+h_int_2

full_ham_matrix = h_kin_mass + h_int

Ham_full = H_kin_mass + H_int_p0 + H_int_m0 + H_int_p1 + H_int_m1 + H_int_p2 + H_int_m2

In [237]:
def check_hermitian(a, rtol=1e-10, atol=1e-10):
    return np.allclose(a, a.conj().T, rtol=rtol, atol=atol)

In [238]:
print(check_hermitian(full_ham_matrix))

True


In [239]:
eigenval,eigenvec = np.linalg.eigh(full_ham_matrix)

In [240]:
print(eigenval)

[-0.57797338 -0.16118742 -0.00948101  0.          0.3         0.3
  0.6         0.6         0.6         0.69051899  0.7         0.9
  1.          1.          1.          1.          1.          1.3
  1.3         1.3         1.4         1.7         1.7         1.86118742
  2.10948101  2.57797338  2.80948101]


In [235]:
print(eigenvec[:,0])

[ 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
  0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 -0.27819651+0.j  0.        +0.j  0.        +0.j  0.        +0.j
  0.        +0.j  0.        +0.j -0.72180349+0.j  0.        +0.j
  0.        +0.j -0.63372425+0.j  0.        +0.j  0.        +0.j
  0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
  0.        +0.j  0.        +0.j  0.        +0.j]


In [217]:
heisenberg = [[1.,i,i+1] for i in range(L-1)]

heisenberg_xy_map = [
    
    ["+-", heisenberg],
    ["-+", heisenberg]
]


heisenberg_z_map = [
    
    ["zz", heisenberg]
]

Ham_xy = hamiltonian(heisenberg_xy_map,dynamic_list=[],basis=basis,**no_checks)
Ham_z = hamiltonian(heisenberg_z_map,dynamic_list=[],basis=basis,**no_checks)




In [284]:
mixing_term = [[0.5,i] for i in range(L)]

mixing_map = [
    
    ["+",mixing_term],
    ["-",mixing_term],
]

Ham_mix = hamiltonian(mixing_map,dynamic_list=[],basis=basis,**no_checks)

In [285]:
resource = [[1.,i,i+1] for i in range(L-2)]

#resource_x0 = [[0.5,0]]

#resource_x = [[-1.,1]]

resource_z = [[1.,2]]


resource_map_1 = [
    
    ["+-", resource],
    ["-+", resource],
]
resource_map_2 = [
    ["+z", resource],
    ["-z", resource],
    ["z", resource_z],
    
]



Ham_res_1 = hamiltonian(resource_map_1,dynamic_list=[],basis=basis,**no_checks)
Ham_res_2 = hamiltonian(resource_map_2,dynamic_list=[],basis=basis,**no_checks)


In [123]:
print(check_hermitian(Ham_res.toarray()))

True


In [124]:
eigenval_mix, eigenvec_mix = Ham_mix.eigh()

In [125]:
#psi_0 = np.zeros(basis.Ns)
#i_0 = basis.index("111")
#psi_0[i_0] = 1.
psi_0 = eigenvec_mix[:,0]

In [286]:
def cost_function(theta,psi,a):

    psi_1 = Ham_mix.evolve(psi,0.0,theta[0])
    psi_2 = Ham_res_1.evolve(psi_1,0.0,theta[1])
    psi_3 = Ham_res_2.evolve(psi_2,0.0,theta[2])
    psi_4 = Ham_mix.evolve(psi_3,0.0,theta[3])
    psi_5 = Ham_res_1.evolve(psi_4,0.0,theta[4])
    psi_6 = Ham_res_2.evolve(psi_5,0.0,theta[5])
    psi_7 = Ham_mix.evolve(psi_6,0.0,theta[6])
    psi_8 = Ham_res_1.evolve(psi_7,0.0,theta[7])
    psi_9 = Ham_res_2.evolve(psi_8,0.0,theta[8])
    psi_10 = Ham_mix.evolve(psi_9,0.0,theta[9])
    psi_11 = Ham_res_1.evolve(psi_10,0.0,theta[10])
    psi_12 = Ham_res_2.evolve(psi_11,0.0,theta[11])

    expt_value = 0
    matvec = full_ham_matrix@psi_12
    for i in range(basis.Ns):
        expt_value += np.conj(psi_12[i])*matvec[i]
    return np.real(expt_value)

In [127]:
print(cost_function(np.array([0.,0.,0.,0.,0.,0.,0.,0.]),psi_0,1))

1.1035533905932724


In [298]:
from scipy.optimize import dual_annealing
import datetime
duan_ranges = [(-10.,10.),(-10.,10.),(-10.,10.),(-10.,10.),(-10.,10.),(-10.,10.),(-10.,10.),(-10.,10.),(-10.,10.),(-10.,10.),(-10.,10.),(-10.,10.)]
resduan = 0
resopt = 0
print(datetime.datetime.now())
resduan = dual_annealing(cost_function, duan_ranges, args = (psi_0,1))
print(datetime.datetime.now())

2022-01-28 12:26:18.880898
2022-01-28 12:31:17.034590


In [294]:
print(resduan.fun)
print(resduan.x)

-0.48798494050152325
[ 2.66879635 -4.57928137  7.56820957  2.53207235 -3.58750107 -8.79564765
 -3.22194073 -1.35827223 -2.3741112   8.38402436  7.44606089 -7.11482358]


In [299]:
print(resduan.fun)
print(resduan.x)

-0.55011331227517
[-4.03878836  5.55989785  1.8595793  -4.86248275  4.78149869 -4.97471836
 -3.46452114 -5.98009609  7.19264618 -2.90166265 -9.05421587  7.69811738]


In [277]:
print(resduan.fun)
print(resduan.x)

-0.40417062801482595
[-0.70515572 -4.70140736 -4.39090833 -4.35340319 -1.77807244 -4.30006263
  8.88176517 -1.67227626  2.2965766   7.44228615  2.19660983  4.52810346]


In [288]:
print(resduan.fun)
print(resduan.x)

-0.4489320776949921
[-0.22209983  3.14402451  4.53690963 -1.85557102 -9.17544668 -7.93384725
 -3.62534467  6.88300907 -3.5392736   2.45358514  9.54778442 -1.64397137]


In [301]:

psi = psi_0
psi_1 = Ham_mix.evolve(psi,0.0,resduan.x[0])
psi_2 = Ham_res_1.evolve(psi_1,0.0,resduan.x[1])
psi_3 = Ham_res_2.evolve(psi_2,0.0,resduan.x[2])
psi_4 = Ham_mix.evolve(psi_3,0.0,resduan.x[3])
psi_5 = Ham_res_1.evolve(psi_4,0.0,resduan.x[4])
psi_6 = Ham_res_2.evolve(psi_5,0.0,resduan.x[5])
psi_7 = Ham_mix.evolve(psi_6,0.0,resduan.x[6])
psi_8 = Ham_res_1.evolve(psi_7,0.0,resduan.x[7])
psi_9 = Ham_res_2.evolve(psi_8,0.0,resduan.x[8])
psi_10 = Ham_mix.evolve(psi_9,0.0,resduan.x[9])
psi_11 = Ham_res_1.evolve(psi_10,0.0,resduan.x[10])
psi_12 = Ham_res_2.evolve(psi_11,0.0,resduan.x[11])

expt_value = 0
matvec = full_ham_matrix@psi_12
for i in range(basis.Ns):
    expt_value += np.conj(psi_12[i])*matvec[i]
print(np.real(expt_value))

-0.55011331227517


In [302]:
print(psi_12)

[-5.63243119e-05+8.09304554e-05j -2.64911789e-03-2.60548801e-03j
  5.67914450e-02-4.09422287e-02j -5.25061577e-06-2.30968812e-05j
  8.91248659e-04-4.85870321e-05j -1.92322669e-03+1.67074816e-02j
  1.71217434e-04-3.42233564e-04j  1.16270866e-02+8.53022662e-03j
 -1.95297808e-01+1.88908513e-01j  2.27977224e-05+1.37020009e-05j
 -6.53468664e-04+7.59972940e-04j -1.20434287e-02-1.45465957e-02j
  3.79530779e-04-9.18874314e-04j  3.17264331e-02+1.99248721e-02j
 -4.70658610e-01+5.26093269e-01j  4.18128388e-04-7.99275053e-04j
  2.70390767e-02+2.06001912e-02j -4.68340276e-01+4.36892518e-01j
  4.52503392e-05-3.54257868e-05j  1.02901402e-03+1.90548756e-03j
 -3.86536434e-02+1.30708946e-02j -1.77740173e-05+1.20485798e-04j
 -4.36291529e-03-1.42421142e-03j  4.02848392e-02-7.65182389e-02j
 -6.83149967e-06-1.68274097e-05j  6.68390289e-04-1.47083738e-04j
  6.12732351e-04+1.28805269e-02j]


In [305]:
scall = 0
for i in range(basis.Ns):
    scall +=np.conj(psi_12[i])*eigenvec[:,0][i]
print(np.abs(scall))

0.990042474045607
